<a href="https://colab.research.google.com/github/sof1a03/DSS_groupproject/blob/main/RDW/RDW_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Car classification



In [112]:
from google.colab import files
import numpy as np
import pandas as pd

In [135]:
rdw_cars = pd.read_csv('df_cleaned_FINAL.csv')
rdw_cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3843 entries, 0 to 3842
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   brand                        3843 non-null   object 
 1   model                        3843 non-null   object 
 2   fuel_types_primary           3843 non-null   object 
 3   resold_flag                  3843 non-null   int64  
 4   inrichting_std               3843 non-null   object 
 5   seats_median                 3843 non-null   float64
 6   mass_empty_median            3843 non-null   float64
 7   length_median                3113 non-null   float64
 8   width_median                 3233 non-null   float64
 9   wheelbase_median             3843 non-null   float64
 10  pw_ratio_median              3843 non-null   float64
 11  datum_eerste_toelating_year  3843 non-null   int64  
 12  count_2023                   3843 non-null   float64
 13  count_2024        

In [136]:
rdw_cars.head()

,brand,model,fuel_types_primary,resold_flag,inrichting_std,seats_median,mass_empty_median,length_median,width_median,wheelbase_median,pw_ratio_median,datum_eerste_toelating_year,count_2023,count_2024,count_2025,avg_2023,avg_2024,avg_2025,image_url_2,image_url_3
0,AUDI,Q4 45 e-tron,Elektriciteit,1,stationwagen,5.0,2135.0,459.0,187.0,277.0,0.000013,2024,105.0,2863.0,1243.0,67735.88,58893.60,57493.41,https://uploads.audi-mediacenter.com/system/pr...,https://uploads.audi-mediacenter.com/system/pr...
1,AUDI,Q2,"Benzine, Diesel",1,stationwagen,5.0,1280.0,421.0,180.0,259.0,0.000058,2023,1891.0,1848.0,940.0,45577.94,46148.48,48594.64,https://uploads.audi-mediacenter.com/system/pr...,https://uploads.audi-mediacenter.com/system/pr...
2,AUDI,A3 Sportback,"Benzine, Diesel, Elektriciteit",1,hatchback,5.0,1255.0,435.0,182.0,262.0,0.000044,2023,3012.0,1837.0,931.0,41083.76,42957.07,43886.43,https://uploads.audi-mediacenter.com/system/pr...,https://daisypstrg.blob.core.windows.net/vehic...
3,AUDI,A1 Sportback,"Benzine, Diesel",1,hatchback,5.0,1080.0,404.0,174.0,255.0,0.000053,2023,2128.0,1626.0,920.0,33815.98,33853.25,36263.14,https://uploads.audi-mediacenter.com/system/pr...,https://daisypstrg.blob.core.windows.net/vehic...
4,AUDI,Q6 Suv e-tron Performance,Elektriciteit,1,stationwagen,5.0,2175.0,477.0,195.0,290.0,0.000022,2025,0.0,329.0,520.0,NaN,82466.87,82587.72,https://media.audi.com/is/image/audi/nemo/mode...,https://uploads.audi-mediacenter.com/system/pr...


## Final cleaning

**Before Doing the classification it was noticed that some sjudtments were needed. Because for now we already have the dataset downloaded with the link we will compute the changes here, for the final this will be cleaned**

1. dropping the rows where brand == model

In [137]:
_brand = rdw_cars["brand"].astype(str).str.strip().str.casefold()
_model = rdw_cars["model"].astype(str).str.strip().str.casefold()

# Mask rows where brand == model (ignoring NaNs safely)
mask_same = _brand.notna() & _model.notna() & (_brand == _model)

# Report and drop in place
n_before = len(rdw_cars)
rdw_cars = rdw_cars.loc[~mask_same].copy()
n_after = len(rdw_cars)

print(f"Dropped {n_before - n_after} rows where brand == model (case-insensitive).")

Dropped 3 rows where brand == model (case-insensitive).


2. droppig duplicates

In [138]:
print(f"Rows before cleaning: {rdw_cars.shape[0]}")
rdw_unique = rdw_cars.drop_duplicates(subset=["brand", "model"], keep="first").copy()
print(f"Rows after cleaning: {rdw_unique.shape[0]}")

Rows before cleaning: 3840
Rows after cleaning: 3725


2.1 removing mistypos

In [139]:
print(f"before: {rdw_unique.shape[0]}")
rdw_unique = rdw_unique[rdw_unique["model"].astype(str).str.strip() != "-"].copy()
print(f"after: {rdw_unique.shape[0]}")

before: 3725
after: 3723


3. Deleting rows where inrichting_std equals “gesloten opbouw” (closed body/box) or “kampeerwagen” (campervan) and exclude all non-passenger body types so they don’t skew passenger-car indicators, clustering, and price/forecast analyses.

In [140]:
rdw_unique["inrichting_std"] = rdw_unique["inrichting_std"].astype(str).str.strip().str.casefold() #consistency

to_exclude = {
    "gesloten opbouw", "kampeerwagen", #afterwards removal
    # broader non-passenger body types
    "open laadvloer", "speciale groep", "neerklapbare zijschotten", "veewagen",
    "voor vervoer voertuigen", "lijkwagen", "huifopbouw", "opleggertrekker",
    "detailhandel/expositiedoel.", "kipper", "gecond. met temperatuurreg.",
    "kraanwagen", "hoogwerker"
}

mask_excl = rdw_unique["inrichting_std"].isin(to_exclude)

n_before = len(rdw_unique)
n_matches = int(mask_excl.sum())
print(f"[BEFORE] Total rows: {n_before}")
print(f"[EXCLUDE] Matching rows (any excluded type): {n_matches}")

if n_matches > 0:
    print("\n[DETAIL] Value counts among rows to be dropped:")
    print(rdw_unique.loc[mask_excl, "inrichting_std"].value_counts())

rdw_unique = rdw_unique.loc[~mask_excl].copy()
n_after = len(rdw_unique)
print(f"\n[AFTER] Total rows: {n_after}")
print(f"Dropped {n_before - n_after} rows.")
print("Remaining unique types (sample):", sorted(rdw_unique["inrichting_std"].dropna().unique())[:12])

[BEFORE] Total rows: 3723
[EXCLUDE] Matching rows (any excluded type): 956

[DETAIL] Value counts among rows to be dropped:
inrichting_std
kampeerwagen                   743
gesloten opbouw                163
neerklapbare zijschotten        15
open laadvloer                  12
opleggertrekker                  6
kipper                           5
lijkwagen                        3
gecond. met temperatuurreg.      3
speciale groep                   1
huifopbouw                       1
detailhandel/expositiedoel.      1
kraanwagen                       1
veewagen                         1
hoogwerker                       1
Name: count, dtype: int64

[AFTER] Total rows: 2767
Dropped 956 rows.
Remaining unique types (sample): ['cabriolet', 'coupe', 'hatchback', 'mpv', 'niet geregistreerd', 'pick-up truck', 'sedan', 'stationwagen', 'voor rolstoelen toegankelijk voertuig']


4. One hot encoding fuel types primary column

   - **1**: the model is produced with that type of fuel,
   
   - **0**: otherwise

In [141]:
# 1) Raw unique values (including NaN)
rdw_unique["fuel_types_primary"].unique()


array(['Elektriciteit', 'Benzine, Diesel',
       'Benzine, Diesel, Elektriciteit', 'Benzine, Elektriciteit',
       'Alcohol, Benzine, Diesel, Elektriciteit', 'Benzine',
       'Alcohol, Benzine, Diesel', 'Benzine, CNG', 'Diesel',
       'Diesel, Elektriciteit', 'CNG', 'Benzine, Diesel, LPG', 'Alcohol',
       'Benzine, Elektriciteit, LPG', 'Benzine, LPG', 'LPG',
       'Alcohol, Benzine, Diesel, LPG',
       'Alcohol, Benzine, Diesel, Elektriciteit, LPG',
       'Benzine, CNG, Elektriciteit, LPG', 'Benzine, CNG, Diesel, LPG',
       'Alcohol, Benzine, LPG', 'Alcohol, Benzine',
       'Benzine, Diesel, Elektriciteit, LPG', 'Elektriciteit, Waterstof',
       'Benzine, CNG, Diesel', 'Benzine, CNG, Diesel, Elektriciteit, LPG',
       'Benzine, CNG, Elektriciteit',
       'Benzine, CNG, Diesel, Elektriciteit',
       'Alcohol, Benzine, CNG, Diesel, Elektriciteit, LPG',
       'Alcohol, Benzine, CNG, Diesel, Elektriciteit'], dtype=object)

In [142]:
if "fuel_types_primary" not in rdw_unique.columns:
    raise KeyError("Column 'fuel_types_primary' not found in rdw_cars")

_map = {
    "benzine": "Benzine",
    "diesel": "Diesel",
    "elektriciteit": "Elektriciteit",
    "lpg": "LPG",
    "cng": "CNG",
    "waterstof": "Waterstof",
    "alcohol": "Alcohol",
}

def parse_fuels(cell):
    if pd.isna(cell):
        return set()
    parts = [p.strip().casefold() for p in str(cell).split(",")]
    return { _map[p] for p in parts if p in _map }


rdw_unique["_fuel_set"] = rdw_unique["fuel_types_primary"].apply(parse_fuels)
fuel_columns = ["Benzine", "Diesel", "Elektriciteit", "LPG", "CNG", "Waterstof"]

# binary colums
for fuel in fuel_columns:
    col_name = f"fuel_{fuel.lower()}"  # e.g., fuel_benzine
    rdw_unique[col_name] = rdw_unique["_fuel_set"].apply(lambda s: int(fuel in s))

print(rdw_unique.filter(regex=r"^fuel_").sum())  # total 1s per fuel column


fuel_types_primary    ElektriciteitBenzine, DieselBenzine, Diesel, E...
fuel_benzine                                                       2327
fuel_diesel                                                         469
fuel_elektriciteit                                                  623
fuel_lpg                                                             62
fuel_cng                                                             23
fuel_waterstof                                                        2
dtype: object


In [143]:
#rdw_unique.drop(columns=["_fuel_set", "fuel_waterstof", "fuel_types_primary"], inplace=True)

6. Z-std the avg price

In [144]:
rdw_unique["price_avg"] = rdw_unique[["avg_2023", "avg_2024", "avg_2025"]].mean(axis=1, skipna=True)

q1, q99 = rdw_unique["price_avg"].quantile([0.01, 0.99])
price_w = rdw_unique["price_avg"].clip(lower=q1, upper=q99)

mu = price_w.mean(skipna=True)
sigma = price_w.std(skipna=True, ddof=0)

if sigma == 0 or np.isnan(sigma):
    rdw_unique["price_z_score"] = np.nan
else:
    rdw_unique["price_z_score"] = (price_w - mu) / sigma

print(f"price_avg non-null: {rdw_unique['price_avg'].notna().sum()} / {len(rdw_unique)}")
print(f"Z-score stats -> mean≈{rdw_unique['price_z_score'].mean(skipna=True):.3f}, std≈{rdw_unique['price_z_score'].std(skipna=True, ddof=0):.3f}")


price_avg non-null: 2767 / 2767
Z-score stats -> mean≈-0.000, std≈1.000


In [145]:
rdw_unique.head(5)

,brand,model,fuel_types_primary,resold_flag,inrichting_std,seats_median,mass_empty_median,length_median,width_median,wheelbase_median,...,image_url_3,_fuel_set,fuel_benzine,fuel_diesel,fuel_elektriciteit,fuel_lpg,fuel_cng,fuel_waterstof,price_avg,price_z_score
0,AUDI,Q4 45 e-tron,Elektriciteit,1,stationwagen,5.0,2135.0,459.0,187.0,277.0,...,https://uploads.audi-mediacenter.com/system/pr...,{Elektriciteit},0,0,1,0,0,0,61374.296667,-0.223644
1,AUDI,Q2,"Benzine, Diesel",1,stationwagen,5.0,1280.0,421.0,180.0,259.0,...,https://uploads.audi-mediacenter.com/system/pr...,"{Benzine, Diesel}",1,1,0,0,0,0,46773.686667,-0.386815
2,AUDI,A3 Sportback,"Benzine, Diesel, Elektriciteit",1,hatchback,5.0,1255.0,435.0,182.0,262.0,...,https://daisypstrg.blob.core.windows.net/vehic...,"{Benzine, Elektriciteit, Diesel}",1,1,1,0,0,0,42642.420000,-0.432985
3,AUDI,A1 Sportback,"Benzine, Diesel",1,hatchback,5.0,1080.0,404.0,174.0,255.0,...,https://daisypstrg.blob.core.windows.net/vehic...,"{Benzine, Diesel}",1,1,0,0,0,0,34644.123333,-0.522371
4,AUDI,Q6 Suv e-tron Performance,Elektriciteit,1,stationwagen,5.0,2175.0,477.0,195.0,290.0,...,https://uploads.audi-mediacenter.com/system/pr...,{Elektriciteit},0,0,1,0,0,0,82527.295000,0.012755


In [146]:
print(f"Initial number of rows: {len(rdw_cars)}")
print(f"Final number of rows: {len(rdw_unique)}")
print(f"Dropped {len(rdw_cars) - len(rdw_unique)} rows.")

Initial number of rows: 3840
Final number of rows: 2767
Dropped 1073 rows.


## Classification

We define keyword lists for specific car categories (e.g., campers, SUVs, MPVs, sports cars).
These keywords allow us to quickly identify vehicles belonging to these categories based on the inrichting_std field.

We calculate the 33rd and 66th percentiles of vehicle mass. These thresholds divide the dataset into three approximate groups — light, medium, and heavy vehicles — which will later help us classify cars into Compact, Medium, or Large categories.

In [147]:
camper_keywords = {"kampeerwagen"}          # (kept for completeness; already excluded above)
suv_keywords    = {"pick-up truck"}
mpv_keywords    = {"mpv", "personenbus"}
sports_keywords = {"cabriolet", "coupe", "sportwagen"}

# Computing the groups based on mass

p33_mass = rdw_unique["mass_empty_median"].quantile(0.33)
p66_mass = rdw_unique["mass_empty_median"].quantile(0.66)
print(f"Global 33rd percentile mass: {p33_mass:.1f}")
print(f"Global 66th percentile mass: {p66_mass:.1f}")

Global 33rd percentile mass: 1128.0
Global 66th percentile mass: 1740.0


This function implements the classification rules. Each vehicle is categorized step-by-step:

Keyword-based rules: Directly assign a class if the description matches a camper, SUV, MPV, or sports car.

Heuristic rules: Use thresholds on mass, length, seat count, or power-to-weight ratio for classification.

Size-based classification: If none of the above applies, vehicles are classified into Compact, Medium, or Large categories based on mass and length.

In [148]:
def classify_car(row):
    inrichting = str(row.get("inrichting_std", "")).casefold()
    mass      = row.get("mass_empty_median", np.nan)
    length    = row.get("length_median", np.nan)
    seats     = row.get("seats_median", np.nan)
    pw_ratio  = row.get("pw_ratio_median", np.nan)

    # Camper
    if inrichting in camper_keywords:
        return "Camper"

    # SUV: keyword OR heavy & long
    if (inrichting in suv_keywords) or (pd.notna(mass) and pd.notna(length) and (mass >= 1800) and (length >= 4500)):
        return "SUV"

    # MPV: keyword OR many seats
    if (inrichting in mpv_keywords) or (pd.notna(seats) and seats >= 6):
        return "MPV"

    # Sports: keyword OR high power-to-weight
    if (inrichting in sports_keywords) or (pd.notna(pw_ratio) and pw_ratio >= 0.12):
        return "Sports"

    # Size-based fallback: Compact / Medium / Large
    if pd.notna(mass) and (mass < p33_mass):
        if pd.notna(length) and (length >= 4500):
            return "Medium"
        else:
            return "Compact"
    elif pd.notna(mass) and (mass < p66_mass):
        return "Medium"
    else:
        if pd.notna(length) and (length < 4400):
            return "Medium"
        else:
            return "Large"

In [149]:
rdw_unique["body_class"] = rdw_unique.apply(classify_car, axis=1)
print("\nBody class distribution:")
print(rdw_unique["body_class"].value_counts(dropna=False))


Body class distribution:
body_class
Medium     1022
Compact     874
Sports      370
MPV         304
SUV         127
Large        70
Name: count, dtype: int64


We apply the classification function to each row of the DataFrame and create a new column body_class.
The distribution of vehicle categories is then displayed, followed by a preview of the classified dataset.

one-hot encoding for body_class column

In [150]:
# No Camper in the categories we encode
_body_classes = ["SUV", "MPV", "Sports", "Compact", "Medium", "Large"]
rdw_unique["body_class"] = pd.Categorical(rdw_unique["body_class"], categories=_body_classes, ordered=False)

# One-hot encode (no 'body_Camper' will be created)
body_dummies = pd.get_dummies(rdw_unique["body_class"], prefix="body", dtype="int8")

# Ensure all expected dummy columns exist (fill 0) and order them
expected_cols = [f"body_{c}" for c in _body_classes]
for col in expected_cols:
    if col not in body_dummies.columns:
        body_dummies[col] = 0
body_dummies = body_dummies[expected_cols]

# Join back
rdw_unique = pd.concat([rdw_unique, body_dummies], axis=1)
#rdw_unique.drop(columns=["body_class"], inplace=True)

# Totals for only the dummy columns (no categorical)
print("\nBody-class dummy totals:")
dummy_cols = [f"body_{c}" for c in _body_classes]
print(rdw_unique[dummy_cols].sum().astype(int))


Body-class dummy totals:
body_SUV         127
body_MPV         304
body_Sports      370
body_Compact     874
body_Medium     1022
body_Large        70
dtype: int64


In [151]:
print(rdw_unique['body_class'].value_counts())
rdw_unique.head(5)

body_class
Medium     1022
Compact     874
Sports      370
MPV         304
SUV         127
Large        70
Name: count, dtype: int64


,brand,model,fuel_types_primary,resold_flag,inrichting_std,seats_median,mass_empty_median,length_median,width_median,wheelbase_median,...,fuel_waterstof,price_avg,price_z_score,body_class,body_SUV,body_MPV,body_Sports,body_Compact,body_Medium,body_Large
0,AUDI,Q4 45 e-tron,Elektriciteit,1,stationwagen,5.0,2135.0,459.0,187.0,277.0,...,0,61374.296667,-0.223644,Medium,0,0,0,0,1,0
1,AUDI,Q2,"Benzine, Diesel",1,stationwagen,5.0,1280.0,421.0,180.0,259.0,...,0,46773.686667,-0.386815,Medium,0,0,0,0,1,0
2,AUDI,A3 Sportback,"Benzine, Diesel, Elektriciteit",1,hatchback,5.0,1255.0,435.0,182.0,262.0,...,0,42642.420000,-0.432985,Medium,0,0,0,0,1,0
3,AUDI,A1 Sportback,"Benzine, Diesel",1,hatchback,5.0,1080.0,404.0,174.0,255.0,...,0,34644.123333,-0.522371,Compact,0,0,0,1,0,0
4,AUDI,Q6 Suv e-tron Performance,Elektriciteit,1,stationwagen,5.0,2175.0,477.0,195.0,290.0,...,0,82527.295000,0.012755,Medium,0,0,0,0,1,0


In [152]:
rdw_unique.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2767 entries, 0 to 3842
Data columns (total 36 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   brand                        2767 non-null   object  
 1   model                        2767 non-null   object  
 2   fuel_types_primary           2767 non-null   object  
 3   resold_flag                  2767 non-null   int64   
 4   inrichting_std               2767 non-null   object  
 5   seats_median                 2767 non-null   float64 
 6   mass_empty_median            2767 non-null   float64 
 7   length_median                2203 non-null   float64 
 8   width_median                 2235 non-null   float64 
 9   wheelbase_median             2767 non-null   float64 
 10  pw_ratio_median              2767 non-null   float64 
 11  datum_eerste_toelating_year  2767 non-null   int64   
 12  count_2023                   2767 non-null   float64 
 13  count_20

In [153]:
# Save the updated DataFrame to a CSV file
rdw_unique.to_csv('rdw_classified.csv', index=False)

files.download('rdw_classified.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>